In [1]:
import cv2
import numpy as np
import matplotlib as plt
import pathlib
from os import listdir
from os.path import isfile, join

In [2]:
# define location of dataset and return all files
dataset_location = "J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021"
target_dir = "I:/FARTS/YOLO_dataset_refined_2048px/"
all_files = [f for f in listdir(dataset_location) if isfile(join(dataset_location, f))]

# next, sort files into images, depth maps, segmentation maps, data, and colony info
dataset_img = []
dataset_depth = []
dataset_seg = []
dataset_data = []
dataset_colony = dataset_location + "/ColonieInfo.csv"

for file in all_files:
    loc = dataset_location + "/" + file
    if file[-7:-4] == "Img":
        dataset_img.append(loc)
    elif file[-7:-4] == "Seg":
        dataset_seg.append(loc)
    elif file[-8:-4] == "Depth":
        dataset_depth.append(loc)
    elif file[-8:-4] == "Data":
        dataset_data.append(loc)
        
print("Found",len(all_files),"files...")

# next sort the colony info into its IDs to determine the colony size and individual scales
# one entry for each successive ID is read
from csv import reader

colony = {'seed': 0,
              'ID': [],
              'scale':[]}

with open(dataset_colony, 'r') as colony_file:
        print("reading", file)
        # pass the file object to reader() to get the reader object
        csv_reader = reader(colony_file)
        # iterate over each row in the csv using reader object
        for r, row in enumerate(csv_reader):
            if r == 0:
                colony['seed'] = row[0].split("=")[-1]
            else:
                colony['ID'].append(row[0].split("=")[-1])
                colony['scale'].append(row[1].split("=")[-1])

print("Loaded colony file with seed",colony['seed'],"and",len(colony['ID']),"individuals.")

Found 39697 files...
reading ColonieInfo.csv
Loaded colony file with seed  0 and 100 individuals.


Now that we have the cleaned colony info, we can start loading the data associated with each frame.
For simplicity we will simply this a list of list as the number of individuals.

We will therefore access "data" as [frame] [individual] [attribute], where attributes will include [ID,bbox_x_0,bbox_y_0,...]

for now training and evaluating detectors, only the bounding box (and ID) info will be relevant

In [3]:
data = []

for file in dataset_data:
    # store all returned coordinates for each individual
    coords = []
        
    # open file in read mode
    with open(file, 'r') as read_obj:
        print("reading", file)
        # pass the file object to reader() to get the reader object
        csv_reader = reader(read_obj)
        # iterate over each row in the csv using reader object
        for row in csv_reader:
            individual = [float(row[0].split(".")[0])]
            # row variable is a list that represents a row in csv
            for elem in row:
                try:
                    individual.append(float(elem.split("=")[-1]))
                except ValueError:
                    pass
            coords.append(individual)
        
    data.append(coords)
    
print("\nThe dataset has a total of", len(data),"generated frames.")

reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/10000_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1000_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1001_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1002_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1003_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1004_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1005_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1006_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1007_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1008_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1009_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/100_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1010_Data.csv
reading J:/FARTS_Project_

reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1106_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1107_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1108_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1109_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/110_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1110_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1111_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1112_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1113_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1114_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1115_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1116_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1117_Data.csv
reading J:/FARTS_Project_U

reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1208_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1209_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/120_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1210_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1211_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1212_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1213_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1214_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1215_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1216_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1217_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1218_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1219_Data.csv
reading J:/FARTS_Project_U

reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1317_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1318_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1319_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/131_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1320_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1321_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1322_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1323_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1324_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1325_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1326_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1327_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1328_Data.csv
reading J:/FARTS_Project_U

reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1417_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1418_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1419_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/141_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1420_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1421_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1422_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1423_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1424_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1425_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1426_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1427_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/1428_Data.csv
reading J:/FARTS_Project_U

...

reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9564_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9565_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9566_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9567_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9568_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9569_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/956_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9570_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9571_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9572_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9573_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9574_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9575_Data.csv
reading J:/FARTS_Project_U

reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9667_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9668_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9669_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/966_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9670_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9671_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9672_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9673_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9674_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9675_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9676_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9677_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9678_Data.csv
reading J:/FARTS_Project_U

reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9770_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9771_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9772_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9773_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9774_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9775_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9776_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9777_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9778_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9779_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/977_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9780_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9781_Data.csv
reading J:/FARTS_Project_U

reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9879_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/987_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9880_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9881_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9882_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9883_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9884_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9885_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9886_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9887_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9888_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9889_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/988_Data.csv
reading J:/FARTS_Project_UE

reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9993_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9994_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9995_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9996_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9997_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9998_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9999_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/999_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/99_Data.csv
reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9_Data.csv

The dataset has a total of 9924 generated frames.


As there may be animals for which we don't use all bones we can return a list of all labels and exclude the respective locations from the pose data. As all animals use the same convention, we can simply read in one example and remove the corresponding indices from all animals.

In [4]:
# first open and read the first line from the first imported data file
labels = []
with open(dataset_data[0], 'r') as read_obj:
    print("reading", file)
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    row_0 = next(csv_reader)  # gets the first line
    # iterate over each row in the csv using reader object
    for elem in row_0:
        try:
            labels.append((elem.split("=")[0].split("Bone.")[-1]))
        except ValueError:
            pass

# now let's define which labels NOT to use (in our case, all labels relating to wings)
omit_labels = ['w_1_r.X', 'w_1_r.Y', 'w_1_r_end.X', 'w_1_r_end.Y', 
               'w_2_r.X', 'w_2_r.Y', 'w_2_r_end.X', 'w_2_r_end.Y',
               'w_1_l.X', 'w_1_l.Y', 'w_1_l_end.X', 'w_1_l_end.Y', 
               'w_2_l.X', 'w_2_l.Y', 'w_2_l_end.X', 'w_2_l_end.Y']

# return indices of labels to be omitted from pose_labels
matched_labels = [i for i, item in enumerate(labels) if item in omit_labels]
print("\nOmitting labels:",omit_labels)
print("\nCorresponding to the following indices:",matched_labels)

# time to remove them from all individuals in "data"
for f, frame in enumerate(data):
    for i, individual in enumerate(frame):
        ind_temp = np.array(data[f][i])
        data[f][i] = np.delete(ind_temp, matched_labels)


# show all used labels:
print("\nAll labels:")
print(labels)

reading J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9_Data.csv

Omitting labels: ['w_1_r.X', 'w_1_r.Y', 'w_1_r_end.X', 'w_1_r_end.Y', 'w_2_r.X', 'w_2_r.Y', 'w_2_r_end.X', 'w_2_r_end.Y', 'w_1_l.X', 'w_1_l.Y', 'w_1_l_end.X', 'w_1_l_end.Y', 'w_2_l.X', 'w_2_l.Y', 'w_2_l_end.X', 'w_2_l_end.Y']

Corresponding to the following indices: [60, 61, 62, 63, 64, 65, 66, 67, 110, 111, 112, 113, 114, 115, 116, 117]

All labels:
['0.BoundingBox.BoundMin.X', 'BoundingBox.BoundMin.Y', 'BoundingBox.BoundMax.X', 'BoundingBox.BoundMax.Y', 'b_t.X', 'b_t.Y', 'b_a_1.X', 'b_a_1.Y', 'b_a_2.X', 'b_a_2.Y', 'b_a_3.X', 'b_a_3.Y', 'b_a_4.X', 'b_a_4.Y', 'b_a_5.X', 'b_a_5.Y', 'b_a_5_end.X', 'b_a_5_end.Y', 'l_1_co_r.X', 'l_1_co_r.Y', 'l_1_tr_r.X', 'l_1_tr_r.Y', 'l_1_fe_r.X', 'l_1_fe_r.Y', 'l_1_ti_r.X', 'l_1_ti_r.Y', 'l_1_ta_r.X', 'l_1_ta_r.Y', 'l_1_pt_r.X', 'l_1_pt_r.Y', 'l_1_pt_r_end.X', 'l_1_pt_r_end.Y', 'l_2_co_r.X', 'l_2_co_r.Y', 'l_2_tr_r.X', 'l_2_tr_r.Y', 'l_2_fe_r.X', 'l_2_fe_r.Y', 'l_2_ti_r.X', 'l_2_ti

Now that we have loaded data and colony info we can start plotting bounding boxes on top of their respective images

In [5]:
# transform between sRGB and linear colour space (optional)

def to_linear(srgb):
    linear = np.float32(srgb) / 255.0
    less = linear <= 0.04045
    linear[less] = linear[less] / 12.92
    linear[~less] = np.power((linear[~less] + 0.055) / 1.055, 2.4)
    return linear * 255.0

    
def from_linear(linear):
    srgb = linear.copy()
    less = linear <= 0.0031308
    srgb[less] = linear[less] * 12.92
    srgb[~less] = 1.055 * np.power(linear[~less], 1.0 / 2.4) - 0.055
    return srgb * 255.0

In [6]:
# create unique colours for each ID
import numpy as np
import time

# alright. Let's take it from the top and fucking multi-thread this.
import threading
import queue
import sys
import os

def getThreads():
    """ Returns the number of available threads on a posix/win based system """
    if sys.platform == 'win32':
        return int(os.environ['NUMBER_OF_PROCESSORS'])
    else:
        return int(os.popen('grep -c cores /proc/cpuinfo').read())

class myThread(threading.Thread):
    def __init__(self, threadID, name, q):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.q = q

    def run(self):
        print("Starting " + self.name)
        process_detections(self.name, self.q)
        print("Exiting " + self.name)
        
def createThreadList(num_threads):
    threadNames = []
    for t in range(num_threads):
        threadNames.append("Thread_" + str(t))

    return threadNames

def process_detections(threadName, q):
    while not exitFlag_stacking:
        queueLock.acquire()
        if not workQueue_stacking.empty():
            
            data_input = q.get()
            i = data_input[0]
            img = data_input[1]
            queueLock.release()
            
            display_img = cv2.imread(img)
            display_img_out = display_img.copy()

            if generate_dataset:
                img_info = []
                
            # compute visibility for each individual
            seg_img = cv2.imread(dataset_seg[i])
            
            individual_visible = False

            for im, individual in enumerate(data[i]):

                fontColor = (int(ID_colours[int(individual[0]),0]),
                             int(ID_colours[int(individual[0]),1]),
                             int(ID_colours[int(individual[0]),2]))
                bbox = fix_bounding_boxes(individual[1:5],max_val=display_img.shape)

                # FOR SOME REASON OCCASIONALLY THE ID OF THE SEG FILE IS LOWER THAN THE DATA FILE
                # with: ID = red_channel/255 * 50
                # red_channel = (ID/50) * 255
                ID_red_val = int((individual[0]/len(colony['ID']))*255)
                try:
                    ID_mask = cv2.inRange(seg_img[bbox[1]:bbox[3],bbox[0]:bbox[2]], np.array([0,0, ID_red_val - 2]), np.array([0,0, ID_red_val + 2]))
                    indivual_occupancy = cv2.countNonZero(ID_mask)
                except:
                    print("Individual fully occluded:",im,"in",dataset_seg[i])
                    indivual_occupancy = 1
                #cv2.imshow("bbox: " + str(im),seg_img[bbox[0]:bbox[2],bbox[1]:bbox[3]])
                #cv2.imshow("mask: " + str(im),ID_mask)
                #cv2.waitKey(0)
                
                #indivual_occupancy = np.count_nonzero((seg_img == [0, 0, int((individual[0]/len(colony['ID']))*255)]).all(axis = 2)) + np.count_nonzero((seg_img == [0, 0, int((individual[0]/len(colony['ID']))*255 - 1)]).all(axis = 2)) + np.count_nonzero((seg_img == [0, 0, int((individual[0]/len(colony['ID']))*255 + 1)]).all(axis = 2))
                bbox_area = abs((bbox[2] - bbox[0]) * (bbox[3] - bbox[1])) + 1
                bbox_occupancy = indivual_occupancy / bbox_area
                #print("Individual", individual[0], "with bounding box occupancy ",bbox_occupancy)

                cv2.putText(display_img, "ID: " + str(int(individual[0])), (bbox[0] + 10,bbox[3] - 10), font, fontScale, fontColor, lineType)
                if bbox_occupancy > visibility_threshold:
                    
                    individual_visible = True
                    """
                    if plot_joints:
                        for point in range(int(len(individual[5:])/2)):
                            cv2.circle(display_img, (int(individual[point*2 + 5]),int(individual[point*2 + 6])), radius=3, color=fontColor, thickness=-1)
                            # let's see of this is really the centre
                            break"""
                            
                    if generate_dataset:
                        # now we need to convert the bounding box info into the desired format.
                        img_dim = display_img.shape

                        # [class_ID, centre_x, centre_y, bounding_box_width, bounding_box_height]
                        class_ID = 0 # here we use a single class, otherwise this can be replaced by size / scale values
                        valid_new_x = False
                        valid_new_y = False
                        
                        if enforce_centred_bboxes:
                            # coords of head
                            b_t = np.array([individual[5], individual[6]])
                            # coords of abdomen
                            b_a_1 = np.array([individual[7], individual[8]])
                            # compute new centre point
                            new_centre_x = (individual[5] + individual[7]) / 2
                            new_centre_y = (individual[6] + individual[8]) / 2
                            
                            if new_centre_x < img_dim[1] and new_centre_x > 0:
                                centre_x = new_centre_x / img_dim[1]
                                valid_new_x = True
                            
                            if new_centre_y < img_dim[0] and new_centre_y > 0:
                                centre_y = new_centre_y / img_dim[0]
                                valid_new_y = True
                            
                            cv2.circle(display_img, (int(new_centre_x),int(new_centre_y)), radius=3, color=fontColor, thickness=-1)    
                        
                        bounding_box_width = abs(bbox[2] - bbox[0]) / img_dim[1]
                        bounding_box_height = abs(bbox[3] - bbox[1]) / img_dim[0]
                        
                        if not valid_new_x or not valid_new_y:
                            centre_x = bbox[0] / img_dim[1] + bounding_box_width / 2
                            centre_y = bbox[1] / img_dim[0] + bounding_box_height / 2
                            
                        img_info.append([class_ID,centre_x,centre_y,bounding_box_width,bounding_box_height])
                        
                        #cv2.rectangle(display_img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), fontColor, 2)
                        
                else:
                    pass
                    # create mask to highlight low visibility animals
                    #blk = np.zeros(display_img.shape, np.uint8)
                    #cv2.rectangle(blk, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 0, 255), cv2.FILLED)

                    # display original bounding box
                    #cv2.rectangle(display_img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), fontColor, 2)
                    # add text to discarded ID
                    #cv2.putText(display_img, "OCCLUDED", (bbox[0] + 10,bbox[3] - 35), font, fontScale, (0,0,255), lineType)

                    # blend the mask with original image
                    #display_img = cv2.addWeighted(display_img, 1.0, blk, 0.25, 1)

                    #print("Individual", int(individual[0]), "has been discarded due to excessive occlusion.")
                    #print("expected:",int((individual[0]/len(colony['ID']))*255))

                if generate_dataset and individual_visible:
                    
                    img_name = target_dir + "/obj/" + img.split('/')[-1][:-4] + "_synth" + ".JPG"
                    cv2.imwrite(img_name, display_img_out)
                    
                    with open(target_dir + "/obj/" + img.split('/')[-1][:-4] + "_synth" + ".txt", "w") as f: 
                        output_txt = []
                        if img_info:
                            for line in img_info:
                                line_str = ' '.join([str(i) for i in line])
                                output_txt.append(line_str+"\n")
                            f.writelines(output_txt)
                        else:
                            f.write("")
                            
            if len(threadList_stacking) == 1:
                cv2.imshow("labeled image", cv2.resize(display_img, (int(display_img.shape[1] / 2), 
                                                                     int(display_img.shape[0] / 2))))
                cv2.waitKey(0)

        else:
            queueLock.release()
            
# setup as many threads as there are (virtual) CPUs
exitFlag_stacking = 0
# only use a fourth of the number of CPUs for stacking as hugin and enfuse utilise multi core processing in part
threadList_stacking = createThreadList(getThreads()*4)
print("Using", len(threadList_stacking), "threads for stacking...")
queueLock = threading.Lock()

# define paths to all images and set the maximum number of items in the queue equivalent to the number of images
workQueue_stacking = queue.Queue(len(dataset_img))
threads = []
threadID = 1


np.random.seed(seed=1)
ID_colours = np.random.randint(255, size=(len(colony['ID']), 3))

font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 0.5
lineType = 2

# we can additionally plot the points in the data files to check joint locations
# WARNING: At the moment there seems to be an issue with inccorrectly given joint locations
plot_joints = False

# remember to refine an export folder when saving out your dataset
generate_dataset = True

# we can enforce the bounding box to centre on the individual instead of being influenced by its orientation
# As the groundtruth in real recordings is annotated in the same way this should boost the average accuracy
enforce_centred_bboxes = True

def fix_bounding_boxes(coords,max_val = [1024,1024]):
    # fix bounding box coordinates so they do not reach beyond the image
    fixed_coords = []
    for c, coord in enumerate(coords):
        if c == 0 or c == 2:
            max_val_temp = max_val[1]
        else:
            max_val_temp = max_val[0]
            
        if coord >= max_val_temp:
            coord = max_val_temp
        elif coord <= 0:
            coord = 0
        
        fixed_coords.append(int(coord))
        
    return fixed_coords

if generate_dataset:
    from Generate_YOLO_training import createCustomFiles
    createCustomFiles(output_folder=target_dir+"/")

# determine the proportion of a bounding box that needs to be filled before considering the visibility as too low
# WARNING: At the moment the ID shown in segmentation maps does not always correspond to the ID in the data file (off by 1)
visibility_threshold = 0.015

timer = time.time()

# Create new threads
for tName in threadList_stacking:
    thread = myThread(threadID, tName, workQueue_stacking)
    thread.start()
    threads.append(thread)
    threadID += 1

# Fill the queue with stacks
queueLock.acquire()
for i,img in enumerate(dataset_img):
    workQueue_stacking.put([i, img])
queueLock.release()

# Wait for queue to empty
while not workQueue_stacking.empty():
    pass

# Notify threads it's time to exit
exitFlag_stacking = 1

# Wait for all threads to complete
for t in threads:
    t.join()
print("Exiting Main Stacking Thread")

# now run createCustomFiles again, to update the train.txt and test.txt files to include the paths to the respective images
if generate_dataset:
    createCustomFiles(output_folder=target_dir+"/")

# close all windows if they were opened
cv2.destroyAllWindows()

print("Total time elapsed:",time.time()-timer,"seconds")

Using 112 threads for stacking...
Using 0 training images and 0 test images. (10.0 %)
Successfully created all required files!
Starting Thread_0
Starting Thread_1
Starting Thread_2
Starting Thread_3
Starting Thread_4
Starting Thread_5
Starting Thread_6
Starting Thread_7
Starting Thread_8
Starting Thread_9
Starting Thread_10
Starting Thread_11
Starting Thread_12
Starting Thread_13
Starting Thread_14
Starting Thread_15
Starting Thread_16
Starting Thread_17
Starting Thread_18
Starting Thread_19
Starting Thread_20
Starting Thread_21
Starting Thread_22
Starting Thread_23
Starting Thread_24
Starting Thread_25
Starting Thread_26
Starting Thread_27
Starting Thread_28
Starting Thread_29
Starting Thread_30
Starting Thread_31
Starting Thread_32
Starting Thread_33
Starting Thread_34
Starting Thread_35
Starting Thread_36
Starting Thread_37
Starting Thread_38
Starting Thread_39
Starting Thread_40
Starting Thread_41
Starting Thread_42
Starting Thread_43
Starting Thread_44
Starting Thread_45
Starting 

Individual fully occluded: 12 in J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/4736_Seg.png
Individual fully occluded: 61 in J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/4718_Seg.png
Individual fully occluded: 83 in J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/473_Seg.png
Individual fully occluded: 2 in J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/4995_Seg.png
Individual fully occluded: 29 in J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/5175_Seg.png
Individual fully occluded: 30 in J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/5189_Seg.png
Individual fully occluded: 46 in J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/5222_Seg.png
Individual fully occluded: 56 in J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/5274_Seg.png
Individual fully occluded: 8 in J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/553_Seg.png
Individual fully occluded: 3 in J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/5682_Seg.png
Individual full

Individual fully occluded: 3 in J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9596_Seg.png
Individual fully occluded: 6 in J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9596_Seg.png
Individual fully occluded: 7 in J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9596_Seg.png
Individual fully occluded: 11 in J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9596_Seg.png
Individual fully occluded: 13 in J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9596_Seg.png
Individual fully occluded: 14 in J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9596_Seg.png
Individual fully occluded: 16 in J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9596_Seg.png
Individual fully occluded: 4 in J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/963_Seg.png
Individual fully occluded: 18 in J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9596_Seg.png
Individual fully occluded: 19 in J:/FARTS_Project_UE5/Output_HD_2048px_10k_11_06_2021/9596_Seg.png
Individual full

Now, re-run **Generate_YOLO_training()** to combine all files into their final test/train sets

In [7]:
from Generate_YOLO_training import createCustomFiles
createCustomFiles(output_folder=target_dir+"/")

Using 8885 training images and 987 test images. (10.0 %)
Successfully created all required files!


In [8]:
# command to train yolo based on generated data:
